In [ ]:
import pickle
import pandas as pd
from collections import defaultdict
import os
import os.path as osp
import numpy as np

In [ ]:
!ls -lha /scratch/SCRATCH_NVME/ilya/pretrain_data/

In [ ]:
for i in os.listdir("/scratch/SCRATCH_NVME/ilya/pretrain_data/"):
    if "pfam" in i:
        uniprot = pd.read_csv("/scratch/SCRATCH_NVME/ilya/pretrain_data/uniprot.tsv.gz", sep='\t').set_index("Entry")[['Protein families', "Cross-reference (Pfam)"]]
        prots = pd.read_pickle("/scratch/SCRATCH_NVME/ilya/pretrain_data/{}".format(i))
        prots.index = prots.index.to_series().apply(lambda x:x.split("-")[1])
        prots = prots.join(uniprot)

        prots = prots[prots['Cross-reference (Pfam)'].notna()].reset_index().drop_duplicates("index").set_index("index")
        prots.drop("Protein families", axis=1, inplace=True)

        prots['fam'] = prots['Cross-reference (Pfam)'].apply(lambda x: x.split(";")[0])
        prots['nnodes'] = prots.data.apply(lambda x: x['x'].size(0))
        prots = prots[prots['nnodes'].between(150, 1000)]

        vc = prots['fam'].value_counts()
        prots = prots[prots['fam'].isin(vc[vc > 10].index)]

        def add_fam(row):
            data = row['data']
            if "index_mapping" in data:
                del data['index_mapping']
            data['fam'] = row['fam']
            return data

        prots['data'] = prots.apply(add_fam, axis=1)
        prots[['data']].to_pickle("/scratch/SCRATCH_NVME/ilya/pretrain_data/{}".format(i.split(".")[0] + "processed.pkl"))

In [ ]:
uniprot = pd.read_csv("/scratch/SCRATCH_NVME/ilya/pretrain_data/uniprot.tsv.gz", sep='\t').set_index("Entry")[['Protein families', "Cross-reference (Pfam)"]]

In [ ]:
uniprot = uniprot[uniprot['Cross-reference (Pfam)'].notna()]

In [ ]:
from collections import defaultdict
counts = defaultdict(set)
for prot_id, fams in uniprot['Cross-reference (Pfam)'].items():
    for fam in fams.rstrip(";").split(";"):
        counts[fam].add(prot_id)

counts = pd.Series({k:len(v) for k,v in counts.items()})

In [ ]:
accepted_fam = counts[counts > 100].index

In [ ]:
def get_new_fams(fams):
    nfam = []
    for fam in fams.rstrip(";").split(";"):
        if fam in accepted_fam:
            nfam.append(fam)
    if nfam:
        return ";".join(nfam)
    return np.nan

In [ ]:
uniprot['f'] = uniprot['Cross-reference (Pfam)'].apply(get_new_fams)

In [ ]:
prots.index = prots.index.to_series().apply(lambda x:x.split("-")[1])
prots = prots.join(uniprot)

In [ ]:
prots = prots[prots['Cross-reference (Pfam)'].notna()].reset_index().drop_duplicates("index").set_index("index")
prots.drop("Protein families", axis=1, inplace=True)

In [ ]:
import pickle
with open("/scratch/SCRATCH_NVME/ilya/pretrain_data/pfam_onehot_onehotprocessed.pkl", "rb") as file:
    prots = pickle.load(file)